In [1]:
import urllib3
from bs4 import BeautifulSoup
import pymongo 
from pprint import pprint 
import pandas as pd
import requests
from splinter import Browser
import json
import tweepy
import pymongo
import os
import time
from pprint import pprint
from selenium import webdriver
from pymongo import MongoClient

In [2]:
def TargetPlayerInfo(rank):
        rank = rank + 1
        
        xpath1 = f'/html/body/table[2]/tbody/tr[{rank}]/td[2]/a'
        xpath2 = f'/html/body/table[2]/tbody/tr[{rank}]/td[3]/span'
        xpath3 = f'/html/body/table[2]/tbody/tr[{rank}]/td[4]/img'
        xpath4 = f'/html/body/table[2]/tbody/tr[{rank}]/td[5]'
        xpath5 = f'/html/body/table[2]/tbody/tr[{rank}]/td[1]'
        
        browser = Browser('chrome', headless=True)
        url = 'https://www.goratings.org/en/'
        browser.visit(url)
        preserve_link = browser.find_by_xpath(xpath1) ## postion as ranking + 1
        
        player_gender = browser.find_by_xpath(xpath2)
        if player_gender.text == '♂':
            gender = "Male"
        else:
            gender = "Female"
        
        player_natinality = browser.find_by_xpath(xpath3)
        
        players_Elo = browser.find_by_xpath(xpath4)
        
        player_rank = browser.find_by_xpath(xpath5)
        
        firstLayer = {
            'Link' : preserve_link['href'],
            'Name' : preserve_link.text,
            "Gender" : gender,
            "Nation" :  player_natinality['alt'], 
            "Elo" : players_Elo.text,
            "Rank": player_rank.text
            
        }
        time.sleep(3)
        
        browser.quit()
        time.sleep(3)
        # start another link scraping
        driver = Browser('chrome', headless=True)
        url1 = firstLayer['Link'] 
        driver.visit(url1)
        html = driver.html
#         soup = BeautifulSoup(html, "html.parser")
        table = driver.find_by_xpath('/html/body/table[1]/tbody')
        info = []
        for i in table.find_by_tag("td"):
            info.append(i.text)


        
        firstLayer["Wins"] = info[0]
        firstLayer["Losses"] = info[1]
        firstLayer["Total"] = info[2]
        firstLayer["Birthday"] = info[-1]
        
        return firstLayer


In [3]:
def GamingRecordDetails(rank):
    
        rank = rank + 1
        
        xpath1 = f'/html/body/table[2]/tbody/tr[{rank}]/td[2]/a'
        browser = Browser('chrome', headless=True)
        url = 'https://www.goratings.org/en/'
        browser.visit(url)
        preserve_link = browser.find_by_xpath(xpath1) ## postion as ranking + 1
        
        SecondLayer = {
            'targeted link' : preserve_link['href'],
            'targeted gamer' : preserve_link.text,
        }
        time.sleep(3)
        
        browser.quit()
       
        time.sleep(3)
        # start another link scraping
        driver = Browser('chrome', headless=True)
        url1 = SecondLayer['targeted link'] 
        driver.visit(url1)
        html = driver.html
        table = driver.find_by_xpath('/html/body/table[2]/tbody')


        info = []

        #table.find_by_tag('td')
        for tr in table.find_by_tag('tr'):
            info.append(tr.text.split(" "))
            
        data = pd.DataFrame(info)

        data = data.drop([0])
        data['Opponant Name'] = data[[4, 5]].apply(lambda x: ' '.join(x), axis=1)
        data = data.drop([4,5], axis=1)
        data
        data = data.rename(columns = {0: "Date", 1: "Gamer Rating", 2: "Color", 3: "Result", 'Opponant Name': "Opponent Name",  6: "Opponent Ranking",
                             7: "Opponent Gender"})

        df = data[["Date", "Gamer Rating","Color", "Result", "Opponent Name",  "Opponent Ranking","Opponent Gender"]]

        df.head()
        final_dict = {

            'Date' : list(df['Date']),
            'Rating': list(df['Gamer Rating']),
            'Color': list(df['Color']),
            'Result': list(df['Result']),
            'Opponent': list(df['Opponent Name']),
            'Op_Ranking': list(df['Opponent Ranking'])

              }
        
        driver.quit()

        return final_dict

In [4]:
# define function to insert record to database
def Add_to_DB(record):
    # Making a Connection with MongoClient 
    client = MongoClient()
    # select/create database
    db = client.GOplayers_db
    # select/create collection
    collection = db.players
    # insert record into collection as a document
    post = collection.insert_one(record)

In [6]:
# scrape top 3 players info and put into mongoDB: players_database
# change to range(1, 51) for top 50 players
for num in range(71, 101):
    # use first two functions to scrape data
    record = TargetPlayerInfo(num)
    games = GamingRecordDetails(num)
    # combine data into one record
    record["Games"] = games
    # insert record into database
    Add_to_DB(record)